# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [2]:
import pandas as pd
import numpy as np
import re
import scipy.stats as stats

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [66]:
NHL = pd.read_csv("nhl.csv")

NHL_2018 = NHL[NHL.year == 2018] # select only the data from 2018

# given by the question
cities = pd.read_html("wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x)) # select the name of the cities

# drop columns with non numeric values from NHL_2018
NHL_2018 = NHL_2018[NHL_2018.GP == "82"]
pattern = r'\*'
NHL_2018 = NHL_2018.replace(pattern, '', regex=True)

# change the columns of the cities df so it does not contain any notes ([note 2], for example)
pattern = r'\[note \d+\]'
cities[["NFL", "MLB", "NBA", 'NHL']] = cities[["NFL", "MLB", "NBA", 'NHL']].replace(pattern, '', regex=True)

cities_NHL = cities.drop(["NFL", "MLB", "NBA"], axis=1)

cities_NHL.set_index("Metropolitan area", inplace=True)
cities_NHL.index = ['New York', 'Los Angeles', 'San Francisco Bay Area', 'Chicago',
       'Dallas', 'Washington', 'Philadelphia', 'Boston',
       'Minessota', 'Colorado', 'Florida', 'Phoenix',
       'Detroit', 'Toronto', 'Houston', 'New Jersey', 'Tampa Bay',
       'Pittsburgh', 'Cleveland', 'Seattle', 'Cincinnati', 'Kansas City',
       'St. Louis', 'Baltimore', 'Charlotte', 'Indianapolis', 'Nashville',
       'Milwaukee', 'New Orleans', 'Buffalo', 'Montreal', 'Vancouver',
       'Orlando', 'Portland', 'Columbus', 'Calgary', 'Ottawa', 'Edmonton',
       'Salt Lake City', 'Winnipeg', 'San Diego', 'San Antonio', 'Sacramento',
       'Vegas', 'Jacksonville', 'Oklahoma City', 'Memphis', 'Carolina',
       'Green Bay', 'Hamilton', 'Regina']

cities_NHL.reset_index(inplace=True)
cities_NHL.set_index("NHL", inplace=True)
cities_NHL.index = ['Rangers, Islanders, Devils', 'Kings, Ducks', 'Sharks', 'Blackhawks',
       'Stars', 'Capitals', 'Flyers', 'Bruins', 'Wild', 'Avalanche',
       'Panthers', 'Coyotes', 'Red Wings', 'Maple Leafs', '—', '', 'Lightning',
       'Penguins', '', '', '—', '', 'Blues', '—', '—', '—', 'Predators', '—',
       '—', 'Sabres', 'Canadiens', 'Canucks', '—', '—', 'Blue Jackets',
       'Flames', 'Senators', 'Oilers', '—', 'Jets', '—', '—', '—',
       'Golden Knights', '—', '—', '—', 'Hurricanes', '—', '', '—']
cities_NHL.reset_index(inplace=True)
cities_NHL.columns = ["Metropolitan area", "NHL", "Population"]
cities_NHL

,Metropolitan area,NHL,Population
0,"Rangers, Islanders, Devils",New York,20153634
1,"Kings, Ducks",Los Angeles,13310447
2,Sharks,San Francisco Bay Area,6657982
3,Blackhawks,Chicago,9512999
4,Stars,Dallas,7233323
5,Capitals,Washington,6131977
6,Flyers,Philadelphia,6070500
7,Bruins,Boston,4794447
8,Wild,Minessota,3551036
9,Avalanche,Colorado,2853077


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.